# EvoGFuzz: An Evolutionary Approach to Grammar-Based Fuzzing

**EvoGFuzz** stands for *evolutionary grammar-based fuzzing*. This approach leverages evolutionary optimization techniques to systematically explore the space of a program's potential inputs, with a particular emphasis on identifying inputs that could lead to exceptional behavior. With a user-defined objective, EvoGFuzz can adapt and refine the input generation strategy over time, making it a powerful tool for uncovering software defects and vulnerabilities.

Efficient detection of defects and vulnerabilities hinges on the ability to automatically generate program inputs that are both valid and diverse. One common strategy is to use grammars, which provide structured and syntactically correct inputs. This approach leads to the concept of grammar-based fuzzing, where fuzzing strategies are guided by the rules defined within the grammar.

A further enhancement to this concept is probabilistic grammar-based fuzzing, where competing grammar rules are associated with probabilities that guide their application. By carefully assigning and optimizing these probabilities, we gain considerable control over the nature of the generated inputs. This enables us to direct the fuzzing process towards specific areas of interest—for example, those functions that are deemed critical, have a higher propensity for failures, or have undergone recent modifications. 

In essence, EvoGFuzz represents a potent blend of evolutionary optimization and probabilistic grammar-based fuzzing, poised to reveal hidden defects and vulnerabilities in a targeted and efficient manner.

## Fuzzing a Program

Our program under investigation is `The Calculator`. This program acts as a typical calculator, capable of evaluating not just arithmetic expressions but also trigonometric functions, such as sine, cosine, and tangent. Furthermore, it also supports the calculation of the square root of a given number.

In [3]:
import math

def calculator(inp: str) -> float:
    """
        A simple calculator function that can evaluate arithmetic expressions 
        and perform basic trigonometric functions and square root calculations.
    """
    return eval(
        str(inp), {"sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan}
    )

**Side Note:** In the `calculator`, we use Python's `eval` function, which takes a string and evaluates it as a Python expression. We provide a dictionary as the second argument to eval, mapping names to corresponding mathematical functions. This enables us to use the function names directly within the input string. 

In [4]:
# Evaluating the cosine of 2π
print(calculator('cos(6*3.141)'))

0.999993677717667


In [5]:
# Calculating the square root of 36
print(calculator('sqrt(6*6)'))

6.0


Each of these calls to the calculator will evaluate the provided string as a mathematical expression, and print the result.

Now, to find new defects, we need to introduce an oracle that tells us if the error that is triggered is something we expect or a new/unkonwn defect. The `OracleResult` is an enum with two possible values, `NO_BUG` and `BUG`. `NO_BUG` donates a passing test case and `BUG` a failing one.

We import the `OracleResult` enumerated type from the `evogfuzz` library. This is used in the oracle function to indicate the outcome of executing the 'calculator' function with a given input.

In [6]:
from evogfuzz.oracle import OracleResult

This is a function called **oracle**, which acts as an intermediary to handle and classify exceptions produced by the calculator function when given a certain input.

In [7]:
# Make sure you use the OracleResult from the evogfuzz library
from evogfuzz.oracle import OracleResult

def oracle(inp: str):
    """
    This function serves as an oracle or intermediary that catches and handles exceptions 
    generated by the 'calculator' function. The oracle function is used in the context of fuzz testing.
    It aims to determine whether an input triggers a bug in the 'calculator' function.

    Args:
        inp (str): The input string to be passed to the 'calculator' function.

    Returns:
        OracleResult: An enumerated type 'OracleResult' indicating the outcome of the function execution.
            - OracleResult.NO_BUG: Returned if the calculator function executes without any exception or only with CalculatorSyntaxError
            - OracleResult.BUG: Returned if the calculator function raises a ValueError exception, indicating a potential bug.
    """
    try:
        calculator(inp)
    except ValueError as e:
        return OracleResult.BUG
    
    return OracleResult.NO_BUG

This **oracle** function is used in the context of fuzzing to determine the impact of various inputs on the program under test (in our case the _calculator_). When the calculator function behaves as expected (i.e., no exceptions occur), the **oracle** function returns `OracleResult.NO_BUG`. However, when the `calculator` function raises an unexpected exception, the **oracle** interprets this as a potential bug in the `calculator` and returns `OracleResult.BUG`.

We can see this in action by testing a few initial inputs:

In [8]:
initial_inputs = ['sqrt(1)', 'cos(912)', 'tan(4)']

for inp in initial_inputs:
    print(inp.ljust(20), oracle(inp))

sqrt(1)              NO_BUG
cos(912)             NO_BUG
tan(4)               NO_BUG


The following code represents a simple context-free grammar for our calculator function. This grammar encompasses all the potential valid inputs to the calculator, which include mathematical expressions involving square roots, trigonometric functions, and integer and decimal numbers:

In [9]:
from fuzzingbook.Grammars import Grammar, is_valid_grammar

CALCGRAMMAR: Grammar = {
    "<start>":
        ["<function>(<term>)"],

    "<function>":
        ["sqrt", "tan", "cos", "sin"],
    
    "<term>": ["-<value>", "<value>"], 
    
    "<value>":
        ["<integer>.<integer>",
         "<integer>"],

    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
}
    
grammar_alhazen: Grammar = {
    "<start>": ["<arith_expr>"],
    "<arith_expr>": ["<function>(<number>)"],
    "<function>": ["sqrt", "sin", "cos", "tan"],
    "<number>": ["<maybe_minus><onenine><maybe_digits><maybe_frac>"],
    "<maybe_minus>": ["", "-"],
    "<onenine>": [str(num) for num in range(1, 10)],
    "<digit>": [str(num) for num in range(0, 10)],
    "<maybe_digits>": ["", "<digits>"],
    "<digits>": ["<digit>", "<digit><digits>"],
    "<maybe_frac>": ["", ".<digits>"],
}
    
assert is_valid_grammar(grammar_alhazen)

The defined grammar CALCGRAMMAR provides a structured blueprint for creating various inputs for our fuzz testing. Each rule in this grammar reflects a possible valid input that our calculator function can handle. By fuzzing based on this grammar, we can systematically explore the space of valid inputs to the calculator function.

### Leveraging EvoGFuzz to Unearth New Defects

We apply our `EvoGFuzz` class to carry out fuzz testing using evolutionary grammar-based fuzzing. This is aimed at uncovering potential defects in our 'calculator' function.

To initialize our EvoGFuzz instance, we require a grammar (in our case, `CALCGRAMMAR`), an oracle function, an initial set of inputs, a fitness function, and the number of iterations to be performed in the fuzzing process.

Upon creating the `EvoGFuzz` instance, we can execute the fuzzing process. The `fuzz()` method runs the fuzzing iterations, evolving the inputs based on our fitness function, and returns a collection of inputs that lead to exceptions in the 'calculator' function.

In [13]:
from evogfuzz.evogfuzz_class import EvoGFuzz

epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    iterations=10
)

Upon creating the `EvoGFuzz` instance, we can execute the fuzzing process. The `.fuzz()` method runs the fuzzing iterations, evolving the inputs based on our fitness function, and returns a collection of inputs that lead to exceptions in the 'calculator' function.

In [14]:
found_exception_inputs = epp.fuzz()
print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

EvoGFuzz found 200 bug-triggering inputs!


Lastly, we can examine the inputs that resulted in exceptions. This output can provide valuable insight into potential weaknesses in the 'calculator' function that need to be addressed.

In [15]:
# print only the first 20 bug-triggering inputs
for inp in list(found_exception_inputs)[:20]:
    print(str(inp))

sqrt(-721)
sqrt(-2747)
sqrt(-6.5)
sqrt(-1635174)
sqrt(-26.777)
sqrt(-9.6)
sqrt(-99.24229)
sqrt(-12.25725)
sqrt(-44562.49251)
sqrt(-72155788731544)
sqrt(-278.2)
sqrt(-214949)
sqrt(-18781.414)
sqrt(-19)
sqrt(-1.72)
sqrt(-4.2472)
sqrt(-4.877599)
sqrt(-6.1)
sqrt(-36531325824471713.53)
sqrt(-64.26)


This process illustrates the power of evolutionary grammar-based fuzzing in identifying new defects within our system. By applying evolutionary algorithms to our fuzzing strategy, we can guide the search towards more defect-prone regions of the input space.

#### Analyzing and Sorting All Generated Inputs by Fitness

After the fuzzing process, you may want to examine all the generated inputs. These can be accessed using the `get_all_inputs()` method. Additionally, we can sort these inputs based on their fitness scores to gain insights into which inputs performed best according to our fitness function.

In [16]:
all_generated_inputs = epp.get_all_inputs()
all_generated_inputs_sorted = sorted(all_generated_inputs, key=lambda inp: inp.fitness, reverse=True)

Now, let's print out these sorted inputs along with their respective fitness scores. Inputs with higher fitness scores will be displayed first, as these are the ones our evolutionary process deemed more likely to uncover potential defects.

In [17]:
# investigate only the first 20 bug-triggering inputs
for inp in all_generated_inputs_sorted[:20]:
    print(f"{str(inp).ljust(40)} fitness: {inp.fitness}")

sqrt(-1635174)                           fitness: 1
sqrt(-9.6)                               fitness: 1
sqrt(-99.24229)                          fitness: 1
sqrt(-44562.49251)                       fitness: 1
sqrt(-1.72)                              fitness: 1
sqrt(-6.1)                               fitness: 1
sqrt(-755)                               fitness: 1
sqrt(-1761.2)                            fitness: 1
sqrt(-46246145.7375)                     fitness: 1
sqrt(-252354574)                         fitness: 1
sqrt(-4)                                 fitness: 1
sqrt(-5587147)                           fitness: 1
sqrt(-488)                               fitness: 1
sqrt(-71256441)                          fitness: 1
sqrt(-117.5)                             fitness: 1
sqrt(-61185)                             fitness: 1
sqrt(-77)                                fitness: 1
sqrt(-6.47)                              fitness: 1
sqrt(-685)                               fitness: 1
sqrt(-56)   

This output provides an overview of the evolved inputs and their effectiveness in revealing potential defects, as gauged by our fitness function. It is a valuable resource for understanding the behavior of our program under various inputs and the effectiveness of our evolutionary grammar-based fuzzing approach.

### Incorporating Custom Fitness Functions

The fitness function plays a crucial role in guiding the evolution process of our fuzzing inputs. A well-crafted fitness function can effectively direct the search towards the most promising regions of the input space.

To create your own fitness function, define a function that takes an `Input` instance and returns a float value. The return value represents the 'fitness' of the given input, with higher values indicating better fitness. Here is a simple template:

```python
from evogfuzz.input import Input

def fitness_function_XYZ(inp: Input) -> float:
    # Implement your fitness function here.
    return 0.0
```

For instance, suppose we're interested in inputs that invoke the cosine function in our calculator. We could define a fitness function `fitness_function_cos` that assigns a high fitness value to inputs containing 'cos'. (**Note that this might not be the best fitness function to find new expcetions.**)

In [18]:
from difflib import SequenceMatcher
from evogfuzz.input import Input

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def fitness_function_XYZ(inp: Input) -> float:
    return similar("cos", str(inp)[:3])

In [19]:
from evogfuzz.input import Input

def fitness_function_cos(inp: Input) -> float:
    if 'cos' in str(inp):
        return 1.0
    else:
        return 0.0

Once your fitness function is defined, you can incorporate it into the `EvoGFuzz` instance by passing it as the `fitness_function` argument. 

In [20]:
epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    fitness_function=fitness_function_XYZ,
    iterations=10
)

found_exception_inputs = epp.fuzz()

print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

for inp in found_exception_inputs:
    print(str(inp))

EvoGFuzz found 13 bug-triggering inputs!
sqrt(-6.912376663)
sqrt(-214)
sqrt(-442)
sqrt(-2.936996927921233921663926167)
sqrt(-114)
sqrt(-17.6197)
sqrt(-61.9)
sqrt(-3372.232339)
sqrt(-1.662)
sqrt(-33216.6171776931)
sqrt(-9)
sqrt(-14)
sqrt(-9.31)


This way, the evolutionary grammar-based fuzzing process is now guided by your custom fitness function, focusing more on the areas you deem critical.

#### Evaluating Inputs Based on Custom Fitness Function

When utilizing a custom fitness function, such as `fitness_function_cos` in our case, we expect inputs containing 'cos' to achieve the highest fitness scores. This is because our fitness function assigns a score of 1.0 to any input that includes 'cos'.

To confirm this behavior, we retrieve all inputs generated during the fuzzing process using the `get_all_inputs()` method and sort these inputs based on their fitness scores.

In [16]:
all_generated_inputs = epp.get_all_inputs()
all_generated_inputs_sorted = sorted(all_generated_inputs, key=lambda inp: inp.fitness, reverse=True)

Let's display these sorted inputs along with their fitness scores. The inputs that contain 'cos' should appear first, demonstrating their high fitness value.

In [17]:
# investigate only the first 20 bug-triggering inputs
for inp in all_generated_inputs_sorted[:20]:
    print(f"{str(inp).ljust(40)} fitness: {inp.fitness}")

cos(14419)                               fitness: 1.0
cos(-5477981.73)                         fitness: 1.0
cos(-892712117.924)                      fitness: 1.0
cos(774544852457797.5)                   fitness: 1.0
cos(-147135233528182187711)              fitness: 1.0
cos(358.752)                             fitness: 1.0
cos(2.12216888)                          fitness: 1.0
cos(2.963)                               fitness: 1.0
cos(-731335.121383)                      fitness: 1.0
cos(3.3)                                 fitness: 1.0
cos(-26613233722156632772621567.7336)    fitness: 1.0
cos(-26638.2)                            fitness: 1.0
cos(-55216558326221813.31)               fitness: 1.0
cos(-88.85)                              fitness: 1.0
cos(-1)                                  fitness: 1.0
cos(-3477524.51722714413813772251813)    fitness: 1.0
cos(1763383718.3178195)                  fitness: 1.0
cos(-6162835263.4611278816511367)        fitness: 1.0
cos(1)                      

In [106]:
from alhazen import feature_collector
from evogfuzz.input import Input
from fuzzingbook.GrammarFuzzer import Grammar
import jaro
import pandas as pd
from sklearn.metrics import pairwise_distances
import fastcluster
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import cosine

inputs_to_feature_vectors = {}
collector = feature_collector.Collector(CALCGRAMMAR)
all_features = collector.get_all_features()

feature_name_2_key = {}
for elem in collector.get_all_features():
    feature_name_2_key[elem.name] = elem.name +" "+ elem.key

for sample in all_generated_inputs_sorted:
    gen_features = collector.collect_features(
        Input(tree=sample.tree
              )
    )
    
    gen_features2 = {}
    for elem in gen_features:
        gen_features2[feature_name_2_key[elem]] = gen_features[elem]
    
    inputs_to_feature_vectors[str(sample)] = gen_features2
    

    
feature_vectors_dataframe = pd.DataFrame.from_dict(inputs_to_feature_vectors).T

distance_matrix= pairwise_distances(feature_vectors_dataframe, metric=cosine)

dendogramm = fastcluster.linkage(distance_matrix, 'ward', preserve_input=False)

num_clust = 100

feature_vectors_index=feature_vectors_dataframe.index
cluster_indexes = fcluster(dendogramm,num_clust, criterion='maxclust')

clusters_sets={}

for index in range(len(feature_vectors_index)):
    clusters_index = cluster_indexes[index]
    input_string = feature_vectors_index[index]
    if clusters_index not in clusters_sets:
        clusters_sets[clusters_index]=set()
    clusters_sets[clusters_index].add(feature_vectors_index[index])
clusters_sets

{27: {'cos(-1141919)',
  'cos(-199112229)',
  'cos(-4219922)',
  'cos(-491424)',
  'cos(-914114)',
  'cos(-9219199)',
  'cos(-94229)',
  'cos(-9499214919224444929911191.9419)',
  'sqrt(-1151553188.1)',
  'sqrt(-118886474933964)',
  'sqrt(-12916)',
  'sqrt(-14199991149994)',
  'sqrt(-16261)',
  'sqrt(-1635174)',
  'sqrt(-1826388261.7)',
  'sqrt(-18781.414)',
  'sqrt(-19125)',
  'sqrt(-214949)',
  'sqrt(-215719.45)',
  'sqrt(-21851)',
  'sqrt(-228271.7)',
  'sqrt(-231344.22)',
  'sqrt(-232746)',
  'sqrt(-24757)',
  'sqrt(-247731)',
  'sqrt(-252354574)',
  'sqrt(-25263)',
  'sqrt(-27255)',
  'sqrt(-3283399844.83974)',
  'sqrt(-36531325824471713.53)',
  'sqrt(-42645256.2)',
  'sqrt(-46246145.7375)',
  'sqrt(-5221.75)',
  'sqrt(-5524524)',
  'sqrt(-5587147)',
  'sqrt(-5657)',
  'sqrt(-5753)',
  'sqrt(-5766.8)',
  'sqrt(-5871.2877)',
  'sqrt(-61185)',
  'sqrt(-61412218761376)',
  'sqrt(-61471.5)',
  'sqrt(-617129)',
  'sqrt(-66182)',
  'sqrt(-6758831)',
  'sqrt(-685577)',
  'sqrt(-686581)',


In [105]:
clusters

array([ 8, 21, 11,  8, 13, 22,  8,  8,  8,  8, 22,  8,  8,  8,  9,  8,  9,
       13,  8,  9, 11,  8,  8, 11,  9,  8,  8,  8,  9,  9, 21,  9,  8,  9,
        8,  8,  8,  8, 11,  8, 22,  9, 24,  8,  8,  7,  8,  8,  9, 12,  8,
       14, 13,  8,  8, 11,  8,  8, 11, 21,  8, 11,  7, 14,  8, 15, 13,  8,
        8,  9, 22, 21,  8, 21, 22,  9, 22,  9,  9, 16,  8, 11,  8,  8,  8,
       21,  8, 11, 21,  9,  8,  9,  8,  9, 22,  8,  8, 24,  8,  8, 21,  8,
        8,  8,  8,  8,  9,  8, 11,  9,  8,  9,  8, 11, 24,  8,  8, 11, 22,
       21,  9,  9,  8,  8,  8,  8, 11, 21,  8,  7, 14,  9,  8, 11,  8,  7,
        7,  8,  8,  7,  9, 21,  8,  8,  7, 13,  8, 23,  7,  7, 22,  8,  7,
       11,  8,  8, 11,  8, 24,  8,  8, 11,  9, 11, 24,  9,  9,  8,  8,  8,
       24,  9, 11,  8,  8, 13,  9,  9,  8,  8,  9,  8,  8, 12,  7,  7,  9,
        9, 13, 14, 14,  8,  8,  8, 22, 21, 22, 11,  8,  8,  2,  1,  8,  4,
        7,  1, 11,  8,  1, 25,  1,  1,  7,  1,  3,  8,  1,  6, 11,  1,  1,
        2,  1,  1,  2,  8

{8: {'cos(-1141919)',
  'cos(-1911)',
  'cos(-199112229)',
  'cos(-4219922)',
  'cos(-491424)',
  'cos(-499)',
  'cos(-914114)',
  'cos(-9219199)',
  'cos(-924)',
  'cos(-941)',
  'cos(-94229)',
  'cos(-9499214919224444929911191.9419)',
  'cos(-994)',
  'sqrt(-1151553188.1)',
  'sqrt(-118886474933964)',
  'sqrt(-12916)',
  'sqrt(-1353)',
  'sqrt(-14199991149994)',
  'sqrt(-16261)',
  'sqrt(-1635174)',
  'sqrt(-1761.2)',
  'sqrt(-1826388261.7)',
  'sqrt(-18781.414)',
  'sqrt(-19125)',
  'sqrt(-1919)',
  'sqrt(-214949)',
  'sqrt(-215719.45)',
  'sqrt(-21851)',
  'sqrt(-2271)',
  'sqrt(-228271.7)',
  'sqrt(-231344.22)',
  'sqrt(-232746)',
  'sqrt(-2374)',
  'sqrt(-2412)',
  'sqrt(-24757)',
  'sqrt(-247731)',
  'sqrt(-252354574)',
  'sqrt(-25263)',
  'sqrt(-27255)',
  'sqrt(-2747)',
  'sqrt(-2921)',
  'sqrt(-3283399844.83974)',
  'sqrt(-36531325824471713.53)',
  'sqrt(-3794.7)',
  'sqrt(-42645256.2)',
  'sqrt(-44562.49251)',
  'sqrt(-46246145.7375)',
  'sqrt(-488)',
  'sqrt(-518)',
  'sqrt

The resulting output validates the effectiveness of our custom fitness function. It shows how we can guide the evolutionary grammar-based fuzzing process towards specific regions of the input space, thereby facilitating targeted exploration and bug discovery.

In [ ]:
import scipy.sparse
import numpy as np

from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine
import  scipy


import fastcluster

from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
import numpy as np

from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import fastcluster


from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
import numpy as np

from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import fastcluster

df=scipy.sparse.coo_matrix((np.array(ertekek),(np.array(sorok),np.array(oszlopok))))

distance_matrix= pairwise_distances(df, metric="cosine")

alenyeg=fastcluster.linkage(distance_matrix, 'ward', preserve_input=False)


num_clust=hanyklaszterlegyen2
kuszob=alenyeg[-num_clust + 1, 2]



from scipy.cluster.hierarchy import fcluster
klaszterek=fcluster(alenyeg, hanyklaszterlegyen2, criterion='maxclust')